In [46]:
# ===== Cell 1: 安裝與導入套件 =====
# 如果缺少套件，請先執行：
!pip install pandas numpy matplotlib seaborn plotly scipy scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
import glob
import warnings
from pathlib import Path
from datetime import datetime
from scipy import stats
import json

warnings.filterwarnings('ignore')

# 設定中文字體（Mac/Linux）
# plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'DejaVu Sans']
# Windows 用戶改用：
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
plt.rcParams['axes.unicode_minus'] = False

# 設定視覺化風格
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("✅ 套件載入完成")
print(f"📍 當前工作目錄: {os.getcwd()}")

✅ 套件載入完成
📍 當前工作目錄: c:\Users\user\Desktop\workspace - 複製 - 複製\orkspace



[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
# ===== Cell 2: 建立專案資料夾結構 =====

def create_project_structure():
    """建立專案所需的資料夾結構"""
    folders = [
        'data/raw',
        'data/processed',
        'reports/figures/raw_timeseries',
        'reports/figures/delta_timeseries',
        'reports/figures/arduino_features',  # 👈 新增：Arduino 特徵視覺化
        'reports/figures/distributions',
        'reports/figures/stability',
        'reports/figures/correlation',
        'reports/figures/sensor_health',     # 👈 新增：感測器健康度
        'reports/summary'
    ]
    
    for folder in folders:
        Path(folder).mkdir(parents=True, exist_ok=True)
    
    print("✅ 資料夾結構建立完成：")
    for folder in folders:
        print(f"   📁 {folder}")

create_project_structure()

✅ 資料夾結構建立完成：
   📁 data/raw
   📁 data/processed
   📁 reports/figures/raw_timeseries
   📁 reports/figures/delta_timeseries
   📁 reports/figures/arduino_features
   📁 reports/figures/distributions
   📁 reports/figures/stability
   📁 reports/figures/correlation
   📁 reports/figures/sensor_health
   📁 reports/summary


In [48]:
!pip install openpyxl


[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
# 測試讀取一個檔案，看看真正的欄位名稱
import pandas as pd
import glob
import os

# 隨便讀一個檔案
test_file = glob.glob('data/raw/*.xlsx')[0]
df_test = pd.read_excel(test_file)

print("📋 實際的欄位名稱：")
print(df_test.columns.tolist())


📋 實際的欄位名稱：
['# ### MODE: PINEAPPLE_INTERNAL_ADC ###', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24']


In [50]:
#發現第一行是註解，所以要寫跳過

In [51]:
# ===== Cell 3: 資料載入器 (Excel 版本) - 支援共用 Air =====

class PineappleDataLoader:
    """批次載入與解析鳳梨感測器數據 (Excel 格式)
    
    支援兩種 air baseline 命名格式：
    1. 專屬 air: pineapple_01_20250205_air.xlsx (給特定鳳梨用)
    2. 共用 air: pineapple_20250205_air.xlsx (多個鳳梨共用)
    """
    
    def __init__(self, data_dir='data/raw'):
        self.data_dir = data_dir
        # air_data：key = "鳳梨編號_日期" 或 "shared_日期"
        self.air_data = {}      
        # pineapple_data：{pineapple_id: {date: DataFrame}}
        self.pineapple_data = {}
        # 共用 air 的映射：{date: DataFrame}
        self.shared_air_data = {}
        
    def load_all_data(self):
        """自動載入所有 Excel 檔案"""
        xlsx_files = glob.glob(os.path.join(self.data_dir, '*.xlsx'))
        
        if not xlsx_files:
            print(f"❌ 在 {self.data_dir} 找不到 Excel 檔案")
            return
        
        print(f"🔍 找到 {len(xlsx_files)} 個 Excel 檔案")
        
        for file_path in xlsx_files:
            filename = os.path.basename(file_path)
            self._parse_and_load(file_path, filename)
        
        print(f"\n✅ 載入完成:")
        print(f"   🌬️  專屬 Air baseline: {len([k for k in self.air_data.keys() if not k.startswith('shared')])}")
        print(f"   🌬️  共用 Air baseline: {len(self.shared_air_data)}")
        print(f"   🍍 Pineapple: {len(self.pineapple_data)} 顆")
        
        return self.air_data, self.pineapple_data, self.shared_air_data
    
    def _parse_and_load(self, file_path, filename):
        """解析單一檔案
        
        支援命名規則：
        1. 專屬 air: pineapple_01_20250205_air.xlsx
        2. 共用 air: pineapple_20250205_air.xlsx (沒有編號)
        3. 鳳梨: pineapple_01_20250205.xlsx
        """
        try:
            parts = filename.replace('.xlsx', '').split('_')
            
            if len(parts) < 2:
                print(f"⚠️  跳過不符合命名規則的檔案: {filename}")
                return
            
            is_air = 'air' in filename
            
            # 判斷是哪種格式
            if is_air:
                if len(parts) == 3:
                    # 格式: pineapple_20250205_air (共用 air)
                    date = parts[1]  # '20250205'
                    pineapple_id = None
                elif len(parts) == 4:
                    # 格式: pineapple_01_20250205_air (專屬 air)
                    pineapple_id = parts[1]  # '01'
                    date = parts[2]  # '20250205'
                else:
                    print(f"⚠️  無法解析 air 檔案格式: {filename}")
                    return
            else:
                if len(parts) >= 3:
                    # 格式: pineapple_01_20250205
                    pineapple_id = parts[1]  # '01'
                    date = parts[2]  # '20250205'
                else:
                    print(f"⚠️  跳過不符合命名規則的檔案: {filename}")
                    return
            
            # 讀取 Excel
            df = self._read_excel_with_validation(file_path)
            
            if df is None:
                return
            
            # 分類儲存
            if is_air:
                if pineapple_id is None:
                    # 共用 air
                    self.shared_air_data[date] = df
                    self.air_data[f"shared_{date}"] = df
                    print(f"   ✓ 共用 Air: {filename} ({len(df)} rows) -> 日期={date}")
                else:
                    # 專屬 air
                    air_key = f"{pineapple_id}_{date}"
                    self.air_data[air_key] = df
                    print(f"   ✓ 專屬 Air: {filename} ({len(df)} rows) -> key={air_key}")
            else:
                if pineapple_id not in self.pineapple_data:
                    self.pineapple_data[pineapple_id] = {}
                self.pineapple_data[pineapple_id][date] = df
                print(f"   ✓ Pineapple: {filename} ({len(df)} rows)")
                
        except Exception as e:
            print(f"❌ 載入失敗 {filename}: {e}")
    
    def _read_excel_with_validation(self, file_path):
        """讀取並驗證 Excel 格式"""
        try:
            # 跳過第 1 行註解，第 2 行是欄位名稱
            df = pd.read_excel(file_path, engine='openpyxl', skiprows=1)
            
            # 驗證必要欄位
            required_cols = [
                'timestamp_ms', 'MQ2_raw', 'MQ3_raw', 'MQ9_raw', 
                'MQ135_raw', 'TGS2602_raw', 'Temp_C', 'Humidity_pct', 'Pressure_hPa'
            ]
            
            missing_cols = [col for col in required_cols if col not in df.columns]
            if missing_cols:
                print(f"⚠️  缺少欄位: {missing_cols}")
                return None
            
            return df
            
        except Exception as e:
            print(f"❌ 讀取錯誤: {e}")
            return None
    
    def get_air_baseline(self, pineapple_id, date):
        """取得對應的 air baseline
        
        優先順序：
        1. 專屬 air (pineapple_id + date)
        2. 共用 air (只有 date)
        3. None (找不到)
        """
        # 先找專屬 air
        air_key = f"{pineapple_id}_{date}"
        if air_key in self.air_data:
            return self.air_data[air_key], "專屬"
        
        # 再找共用 air
        if date in self.shared_air_data:
            return self.shared_air_data[date], "共用"
        
        return None, "無"

# 測試載入
loader = PineappleDataLoader('data/raw')
air_data, pineapple_data, shared_air_data = loader.load_all_data()

# 測試 air baseline 查詢
print("\n🔍 測試 Air Baseline 查詢:")
for pid in ['01', '07', '08']:
    for date in ['20250203', '20250204', '20250205']:
        air_df, air_type = loader.get_air_baseline(pid, date)
        if air_df is not None:
            print(f"   🍍 {pid} @ {date}: {air_type} air ({len(air_df)} rows)")
        else:
            print(f"   ❌ {pid} @ {date}: 找不到 air baseline")


🔍 找到 9 個 Excel 檔案
   ✓ Pineapple: pineapple_07_20250203.xlsx (912 rows)
   ✓ Pineapple: pineapple_07_20250204.xlsx (912 rows)
   ✓ Pineapple: pineapple_07_20250205.xlsx (913 rows)
   ✓ Pineapple: pineapple_08_20250203.xlsx (912 rows)
   ✓ Pineapple: pineapple_08_20250204.xlsx (912 rows)
   ✓ Pineapple: pineapple_08_20250205.xlsx (912 rows)
   ✓ 共用 Air: pineapple_20250203_air.xlsx (912 rows) -> 日期=20250203
   ✓ 共用 Air: pineapple_20250204_air.xlsx (912 rows) -> 日期=20250204
   ✓ 共用 Air: pineapple_20250205_air.xlsx (912 rows) -> 日期=20250205

✅ 載入完成:
   🌬️  專屬 Air baseline: 0
   🌬️  共用 Air baseline: 3
   🍍 Pineapple: 2 顆

🔍 測試 Air Baseline 查詢:
   🍍 01 @ 20250203: 共用 air (912 rows)
   🍍 01 @ 20250204: 共用 air (912 rows)
   🍍 01 @ 20250205: 共用 air (912 rows)
   🍍 07 @ 20250203: 共用 air (912 rows)
   🍍 07 @ 20250204: 共用 air (912 rows)
   🍍 07 @ 20250205: 共用 air (912 rows)
   🍍 08 @ 20250203: 共用 air (912 rows)
   🍍 08 @ 20250204: 共用 air (912 rows)
   🍍 08 @ 20250205: 共用 air (912 rows)


In [52]:
# ===== Cell 4: 資料清洗器 + 感測器健康度檢查 =====

class DataCleaner:
    """資料清洗與驗證"""
    
    def __init__(self):
        # 定義異常值範圍（根據你的 Arduino code）
        self.validation_rules = {
            'MQ2_raw': (10, 1000),
            'MQ3_raw': (10, 1000),
            'MQ9_raw': (10, 1000),
            'MQ135_raw': (10, 1000),
            'TGS2602_raw': (10, 1000),
            'Temp_C': (0, 50),
            'Humidity_pct': (0, 100),
            'Pressure_hPa': (800, 1100)
        }
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
    
    def clean_dataset(self, df, dataset_name="Dataset"):
        """清洗單一數據集"""
        print(f"\n🧹 清洗 {dataset_name}...")
        
        original_rows = len(df)
        
        # 1. 移除完全重複的行
        df = df.drop_duplicates()
        
        # 2. 檢查並處理缺值
        missing_before = df.isnull().sum().sum()
        if missing_before > 0:
            print(f"   ⚠️  發現 {missing_before} 個缺值")
            df = df.dropna().reset_index(drop=True)  # 👈 關鍵：重置 index
        
        # 3. 異常值過濾
        mask = pd.Series([True] * len(df))
        outlier_counts = {}
        
        for col, (min_val, max_val) in self.validation_rules.items():
            if col in df.columns:
                col_mask = (df[col] >= min_val) & (df[col] <= max_val)
                outliers = (~col_mask).sum()
                if outliers > 0:
                    outlier_counts[col] = outliers
                mask &= col_mask
        
        df_clean = df[mask].copy()
        
        # 4. 時間對齊（重設 timestamp_s 從 0 開始）
        if 'timestamp_ms' in df_clean.columns:
            df_clean['timestamp_s'] = (df_clean['timestamp_ms'] - df_clean['timestamp_ms'].iloc[0]) / 1000.0
        
        # 5. 感測器健康度檢查
        health_report = self.check_sensor_health(df_clean, dataset_name)
        
        # 報告
        removed_rows = original_rows - len(df_clean)
        print(f"   ✓ 原始: {original_rows} rows")
        print(f"   ✓ 移除: {removed_rows} rows ({removed_rows/original_rows*100:.2f}%)")
        print(f"   ✓ 保留: {len(df_clean)} rows")
        
        if outlier_counts:
            print(f"   📊 異常值統計:")
            for col, count in outlier_counts.items():
                print(f"      - {col}: {count}")
        
        return df_clean, {
            'original_rows': original_rows,
            'removed_rows': removed_rows,
            'outlier_counts': outlier_counts,
            'final_rows': len(df_clean),
            'sensor_health': health_report
        }
    
    def check_sensor_health(self, df, dataset_name):
        """檢查感測器健康度"""
        print(f"   🔬 感測器健康度:")
        
        health_report = {}
        
        for col in self.sensor_cols:
            if col in df.columns:
                mean_val = df[col].mean()
                std_val = df[col].std()
                cv = (std_val / mean_val * 100) if mean_val != 0 else 0
                
                # 檢查顯著跳變（> 5 個標準差）
                diff = df[col].diff().abs()
                significant_jumps = (diff > std_val * 5).sum() if std_val > 0 else 0
                
                # 評級
                if cv < 1:
                    grade = "⚠️ 極穩定（可能未接觸氣體）"
                elif cv < 5:
                    grade = "✓ 穩定"
                elif cv < 15:
                    grade = "✓ 正常"
                else:
                    grade = "⚠️ 變化較大"
                
                health_report[col] = {
                    'cv_percent': cv,
                    'significant_jumps': significant_jumps,
                    'grade': grade
                }
                
                print(f"      {col}: CV={cv:.2f}%, 顯著跳變={significant_jumps}次, {grade}")
        
        return health_report
    
    def clean_all_data(self, air_data, pineapple_data):
        """批次清洗所有數據"""
        print("="*50)
        print("🧹 開始批次清洗數據")
        print("="*50)
        
        cleaned_air = {}
        cleaned_pineapple = {}
        cleaning_report = {'air': {}, 'pineapple': {}}
        
        # 清洗 air baseline
        for pid, df in air_data.items():
            cleaned_air[pid], cleaning_report['air'][pid] = \
                self.clean_dataset(df, f"Air_{pid}")
        
        # 清洗 pineapple
        for pid, date_dict in pineapple_data.items():
            cleaned_pineapple[pid] = {}
            cleaning_report['pineapple'][pid] = {}
            for date, df in date_dict.items():
                cleaned_pineapple[pid][date], cleaning_report['pineapple'][pid][date] = \
                    self.clean_dataset(df, f"Pineapple_{pid}_{date}")
        
        print("\n✅ 清洗完成！")
        return cleaned_air, cleaned_pineapple, cleaning_report

# 執行清洗
cleaner = DataCleaner()
cleaned_air, cleaned_pineapple, cleaning_report = cleaner.clean_all_data(air_data, pineapple_data)

🧹 開始批次清洗數據

🧹 清洗 Air_shared_20250203...
   ⚠️  發現 288 個缺值
   🔬 感測器健康度:
      MQ2_raw: CV=2.91%, 顯著跳變=0次, ✓ 穩定
      MQ3_raw: CV=4.54%, 顯著跳變=0次, ✓ 穩定
      MQ9_raw: CV=1.31%, 顯著跳變=0次, ✓ 穩定
      MQ135_raw: CV=2.56%, 顯著跳變=0次, ✓ 穩定
      TGS2602_raw: CV=12.00%, 顯著跳變=0次, ✓ 正常
   ✓ 原始: 912 rows
   ✓ 移除: 12 rows (1.32%)
   ✓ 保留: 900 rows

🧹 清洗 Air_shared_20250204...
   ⚠️  發現 288 個缺值
   🔬 感測器健康度:
      MQ2_raw: CV=2.90%, 顯著跳變=0次, ✓ 穩定
      MQ3_raw: CV=7.10%, 顯著跳變=0次, ✓ 正常
      MQ9_raw: CV=2.97%, 顯著跳變=0次, ✓ 穩定
      MQ135_raw: CV=2.93%, 顯著跳變=0次, ✓ 穩定
      TGS2602_raw: CV=14.45%, 顯著跳變=0次, ✓ 正常
   ✓ 原始: 912 rows
   ✓ 移除: 12 rows (1.32%)
   ✓ 保留: 900 rows

🧹 清洗 Air_shared_20250205...
   ⚠️  發現 288 個缺值
   🔬 感測器健康度:
      MQ2_raw: CV=3.15%, 顯著跳變=0次, ✓ 穩定
      MQ3_raw: CV=4.09%, 顯著跳變=0次, ✓ 穩定
      MQ9_raw: CV=5.73%, 顯著跳變=0次, ✓ 正常
      MQ135_raw: CV=3.67%, 顯著跳變=0次, ✓ 穩定
      TGS2602_raw: CV=15.91%, 顯著跳變=0次, ⚠️ 變化較大
   ✓ 原始: 912 rows
   ✓ 移除: 12 rows (1.32%)
   ✓ 保留: 900 rows

🧹 清洗 Pineapple_0

In [53]:
# ===== Cell 5: Delta 計算器（支援共用 Air）=====

class DeltaCalculator:
    """計算 Pineapple 與 Air baseline 的差異"""
    
    def __init__(self, cleaned_air, cleaned_pineapple):
        self.cleaned_air = cleaned_air
        self.cleaned_pineapple = cleaned_pineapple
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
        self.delta_data = {}
    
    def calculate_all_deltas(self):
        """批次計算所有 Delta"""
        print("📐 計算 Delta (Pineapple - Air)...")
        
        for pid, date_dict in self.cleaned_pineapple.items():
            self.delta_data[pid] = {}
            
            for date, pineapple_df in date_dict.items():
                # 🔧 修改：優先找專屬 air，找不到用共用 air
                air_df = self._get_air_baseline(pid, date)
                
                if air_df is None:
                    print(f"   ⚠️  Pineapple_{pid}_{date} 找不到對應的 Air baseline，跳過")
                    continue
                
                # 計算 Delta
                delta_df = self._calculate_delta(pineapple_df, air_df, pid, date)
                
                if delta_df is not None:
                    self.delta_data[pid][date] = delta_df
                    print(f"   ✓ Pineapple_{pid}_{date}: {len(delta_df)} rows")
        
        print("✅ Delta 計算完成！")
        return self.delta_data
    
    def _get_air_baseline(self, pineapple_id, date):
        """
        取得對應的 air baseline
        優先順序：
        1. 專屬 air (pineapple_id_date)
        2. 共用 air (shared_date)
        """
        # 先找專屬 air
        air_key = f"{pineapple_id}_{date}"
        if air_key in self.cleaned_air:
            return self.cleaned_air[air_key]
        
        # 找不到就找共用 air
        shared_key = f"shared_{date}"
        if shared_key in self.cleaned_air:
            return self.cleaned_air[shared_key]
        
        return None
    
    def _calculate_delta(self, pineapple_df, air_df, pid, date):
        """計算單一數據集的 Delta"""
        try:
            # 對齊時間軸（取較短的長度）
            min_len = min(len(pineapple_df), len(air_df))
            pineapple_aligned = pineapple_df.iloc[:min_len].reset_index(drop=True)
            air_aligned = air_df.iloc[:min_len].reset_index(drop=True)
            
            # 計算 Delta
            delta_df = pineapple_aligned.copy()
            
            for col in self.sensor_cols:
                if col in pineapple_aligned.columns and col in air_aligned.columns:
                    delta_df[f'{col}_delta'] = pineapple_aligned[col] - air_aligned[col]
            
            return delta_df
            
        except Exception as e:
            print(f"   ❌ 計算 Delta 失敗 {pid}_{date}: {e}")
            return None

# 執行 Delta 計算
calculator = DeltaCalculator(cleaned_air, cleaned_pineapple)
delta_data = calculator.calculate_all_deltas()

# 顯示結果
print(f"\n📊 Delta 數據統計:")
for pid, date_dict in delta_data.items():
    print(f"   🍍 {pid}: {len(date_dict)} 天的數據")




📐 計算 Delta (Pineapple - Air)...
   ✓ Pineapple_07_20250203: 900 rows
   ✓ Pineapple_07_20250204: 900 rows
   ✓ Pineapple_07_20250205: 899 rows
   ✓ Pineapple_08_20250203: 900 rows
   ✓ Pineapple_08_20250204: 900 rows
   ✓ Pineapple_08_20250205: 900 rows
✅ Delta 計算完成！

📊 Delta 數據統計:
   🍍 07: 3 天的數據
   🍍 08: 3 天的數據


In [54]:
# ===== Cell 6: 視覺化 - Raw 時序圖 =====

class TimeSeriesVisualizer:
    """時序圖視覺化"""
    
    def __init__(self, output_dir='reports/figures'):
        self.output_dir = output_dir
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
        self.env_cols = ['Temp_C', 'Humidity_pct', 'Pressure_hPa']
        
    def plot_raw_timeseries(self, df, title, save_path=None):
        """繪製 raw 值時序圖（8 subplots）"""
        fig = make_subplots(
            rows=3, cols=3,
            subplot_titles=[
                'MQ2 (Smoke)', 'MQ3 (Alcohol)', 'MQ9 (CO/CH4)',
                'MQ135 (NH3/CO2)', 'TGS2602 (VOC)', 'Temperature',
                'Humidity', 'Pressure', ''
            ],
            vertical_spacing=0.08,
            horizontal_spacing=0.08
        )
        
        time_col = 'timestamp_s' if 'timestamp_s' in df.columns else df.index
        
        # 感測器數據
        sensors = [
            ('MQ2_raw', 1, 1, 'blue'),
            ('MQ3_raw', 1, 2, 'red'),
            ('MQ9_raw', 1, 3, 'green'),
            ('MQ135_raw', 2, 1, 'purple'),
            ('TGS2602_raw', 2, 2, 'orange'),
            ('Temp_C', 2, 3, 'darkred'),
            ('Humidity_pct', 3, 1, 'cyan'),
            ('Pressure_hPa', 3, 2, 'brown')
        ]
        
        for col, row, col_pos, color in sensors:
            if col in df.columns:
                fig.add_trace(
                    go.Scatter(
                        x=df[time_col] if isinstance(time_col, str) else time_col,
                        y=df[col],
                        mode='lines',
                        name=col,
                        line=dict(color=color, width=1.5),
                        showlegend=False
                    ),
                    row=row, col=col_pos
                )
        
        fig.update_xaxes(title_text="Time (s)", row=3)
        fig.update_yaxes(title_text="ADC Value", row=1, col=1)
        fig.update_yaxes(title_text="ADC Value", row=2, col=1)
        fig.update_yaxes(title_text="°C", row=2, col=3)
        fig.update_yaxes(title_text="%", row=3, col=1)
        fig.update_yaxes(title_text="hPa", row=3, col=2)
        
        fig.update_layout(
            title_text=title,
            height=900,
            showlegend=False,
            template='plotly_white'
        )
        
        if save_path:
            fig.write_html(save_path)
            print(f"   💾 已儲存: {save_path}")
        
        return fig
    
    def plot_all_raw_timeseries(self, cleaned_air, cleaned_pineapple):
        """批次繪製所有 raw 時序圖"""
        print("\n📈 繪製 Raw 時序圖...")
        
        # Air baseline
        for pid, df in cleaned_air.items():
            save_path = f"{self.output_dir}/raw_timeseries/air_{pid}_raw.html"
            self.plot_raw_timeseries(df, f"Air Baseline - {pid}", save_path)
        
        # Pineapple
        for pid, date_dict in cleaned_pineapple.items():
            for date, df in date_dict.items():
                save_path = f"{self.output_dir}/raw_timeseries/pineapple_{pid}_{date}_raw.html"
                self.plot_raw_timeseries(df, f"Pineapple {pid} - {date}", save_path)
        
        print("✅ Raw 時序圖繪製完成！")

# 執行視覺化
visualizer = TimeSeriesVisualizer()
visualizer.plot_all_raw_timeseries(cleaned_air, cleaned_pineapple)


📈 繪製 Raw 時序圖...
   💾 已儲存: reports/figures/raw_timeseries/air_shared_20250203_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_shared_20250204_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_shared_20250205_raw.html
   💾 已儲存: reports/figures/raw_timeseries/pineapple_07_20250203_raw.html
   💾 已儲存: reports/figures/raw_timeseries/pineapple_07_20250204_raw.html
   💾 已儲存: reports/figures/raw_timeseries/pineapple_07_20250205_raw.html
   💾 已儲存: reports/figures/raw_timeseries/pineapple_08_20250203_raw.html
   💾 已儲存: reports/figures/raw_timeseries/pineapple_08_20250204_raw.html
   💾 已儲存: reports/figures/raw_timeseries/pineapple_08_20250205_raw.html
✅ Raw 時序圖繪製完成！


In [55]:
# ===== Cell 7.5: Arduino 特徵提取器 =====

class ArduinoFeatureExtractor:
    """
    提取 Arduino 式特徵（Rs/R0, PPM 估算等）
    模擬 Arduino 上的氣體濃度計算邏輯
    """
    
    def __init__(self, delta_data):
        self.delta_data = delta_data
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
        self.arduino_features = {}
        
        # R0 校正參考值（在乾淨空氣中測得，可根據實際調整）
        self.R0_reference = {
            'MQ2_raw': 10.0,
            'MQ3_raw': 60.0,
            'MQ9_raw': 9.8,
            'MQ135_raw': 76.63,
            'TGS2602_raw': 2.5
        }
    
    def extract_all_features(self):
        """批次提取所有 Arduino 特徵"""
        print("🔧 提取 Arduino 特徵...")
        
        for pid, date_dict in self.delta_data.items():
            self.arduino_features[pid] = {}
            
            for date, delta_df in date_dict.items():
                features_df = self._calculate_arduino_features(delta_df, pid, date)
                
                if features_df is not None:
                    self.arduino_features[pid][date] = features_df
                    print(f"   ✓ Pineapple_{pid}_{date}: 提取 {len(features_df.columns)} 個特徵")
        
        print("✅ Arduino 特徵提取完成！")
        return self.arduino_features
    
    def _calculate_arduino_features(self, delta_df, pid, date):
        """計算單一數據集的 Arduino 特徵"""
        try:
            features_df = delta_df.copy()
            
            # 計算 Rs/R0 (感測器電阻比)
            for col in self.sensor_cols:
                if col in delta_df.columns:
                    sensor_name = col.replace('_raw', '')
                    R0 = self.R0_reference.get(col, 10.0)
                    
                    # Rs/R0 = (raw_value / max_adc) * (Vc / RL) / R0
                    # 簡化版：直接用 raw / R0 作為比例
                    features_df[f'{sensor_name}_Rs_R0'] = delta_df[col] / R0
                    
                    # 計算 delta 的 Rs/R0（變化量）
                    delta_col = f'{col}_delta'
                    if delta_col in delta_df.columns:
                        features_df[f'{sensor_name}_delta_Rs_R0'] = delta_df[delta_col] / R0
            
            # 計算統計特徵
            for col in self.sensor_cols:
                if col in delta_df.columns:
                    sensor_name = col.replace('_raw', '')
                    
                    # 移動平均（smoothing）
                    window = 10
                    features_df[f'{sensor_name}_ma{window}'] = delta_df[col].rolling(window=window, center=True).mean()
                    
                    # 移動標準差（穩定性）
                    features_df[f'{sensor_name}_std{window}'] = delta_df[col].rolling(window=window, center=True).std()
            
            return features_df
            
        except Exception as e:
            print(f"   ❌ 提取特徵失敗 {pid}_{date}: {e}")
            return None

# 執行 Arduino 特徵提取
arduino_extractor = ArduinoFeatureExtractor(delta_data)
arduino_features = arduino_extractor.extract_all_features()

# 顯示結果
print(f"\n📊 Arduino 特徵統計:")
for pid, date_dict in arduino_features.items():
    print(f"   🍍 {pid}: {len(date_dict)} 天的數據")
    for date, features_df in date_dict.items():
        print(f"      - {date}: {features_df.shape[1]} 個特徵欄位")




🔧 提取 Arduino 特徵...
   ✓ Pineapple_07_20250203: 提取 50 個特徵
   ✓ Pineapple_07_20250204: 提取 50 個特徵
   ✓ Pineapple_07_20250205: 提取 50 個特徵
   ✓ Pineapple_08_20250203: 提取 50 個特徵
   ✓ Pineapple_08_20250204: 提取 50 個特徵
   ✓ Pineapple_08_20250205: 提取 50 個特徵
✅ Arduino 特徵提取完成！

📊 Arduino 特徵統計:
   🍍 07: 3 天的數據
      - 20250203: 50 個特徵欄位
      - 20250204: 50 個特徵欄位
      - 20250205: 50 個特徵欄位
   🍍 08: 3 天的數據
      - 20250203: 50 個特徵欄位
      - 20250204: 50 個特徵欄位
      - 20250205: 50 個特徵欄位


In [56]:
# ===== Cell 8: Arduino 特徵視覺化 =====

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

class ArduinoFeaturePlotter:
    """繪製 Arduino 特徵圖表"""
    
    def __init__(self, arduino_features):
        self.arduino_features = arduino_features
        self.output_dir = 'reports/figures/arduino_features'
        os.makedirs(self.output_dir, exist_ok=True)
    
    def plot_all(self):
        """繪製所有 Arduino 特徵圖"""
        print("🎨 繪製 Arduino 特徵圖...")
        
        if not self.arduino_features:
            print("⚠️  未找到 Arduino 特徵數據")
            return
        
        for pid, date_dict in self.arduino_features.items():
            for date, features_df in date_dict.items():
                self._plot_arduino_features(features_df, pid, date)
        
        print("✅ Arduino 特徵圖繪製完成！")
    
    def _plot_arduino_features(self, features_df, pid, date):
        """繪製單一數據集的 Arduino 特徵"""
        try:
            # 建立 subplots
            fig = make_subplots(
                rows=3, cols=1,
                subplot_titles=(
                    f'🔧 Rs/R0 比值 - Pineapple {pid} ({date})',
                    f'📊 Delta Rs/R0 - Pineapple {pid} ({date})',
                    f'📈 移動平均（MA10）- Pineapple {pid} ({date})'
                ),
                vertical_spacing=0.08
            )
            
            sensors = ['MQ2', 'MQ3', 'MQ9', 'MQ135', 'TGS2602']
            colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
            
            # 第 1 行：Rs/R0
            for sensor, color in zip(sensors, colors):
                col_name = f'{sensor}_Rs_R0'
                if col_name in features_df.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=features_df.index,
                            y=features_df[col_name],
                            mode='lines',
                            name=sensor,
                            line=dict(color=color, width=2),
                            legendgroup='rs_r0',
                            showlegend=True
                        ),
                        row=1, col=1
                    )
            
            # 第 2 行：Delta Rs/R0
            for sensor, color in zip(sensors, colors):
                col_name = f'{sensor}_delta_Rs_R0'
                if col_name in features_df.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=features_df.index,
                            y=features_df[col_name],
                            mode='lines',
                            name=sensor,
                            line=dict(color=color, width=2),
                            legendgroup='delta_rs_r0',
                            showlegend=False
                        ),
                        row=2, col=1
                    )
            
            # 第 3 行：移動平均
            for sensor, color in zip(sensors, colors):
                col_name = f'{sensor}_ma10'
                if col_name in features_df.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=features_df.index,
                            y=features_df[col_name],
                            mode='lines',
                            name=sensor,
                            line=dict(color=color, width=2),
                            legendgroup='ma10',
                            showlegend=False
                        ),
                        row=3, col=1
                    )
            
            # 更新佈局
            fig.update_layout(
                height=900,
                title_text=f"🔧 Arduino 特徵分析 - Pineapple {pid} ({date})",
                showlegend=True,
                hovermode='x unified'
            )
            
            fig.update_xaxes(title_text="樣本點", row=3, col=1)
            fig.update_yaxes(title_text="Rs/R0", row=1, col=1)
            fig.update_yaxes(title_text="Delta Rs/R0", row=2, col=1)
            fig.update_yaxes(title_text="Raw Value (MA10)", row=3, col=1)
            
            # 儲存
            filename = f"pineapple_{pid}_{date}_arduino.html"
            filepath = os.path.join(self.output_dir, filename)
            fig.write_html(filepath)
            print(f"   💾 已儲存: {filepath}")
            
        except Exception as e:
            print(f"   ❌ 繪圖失敗 {pid}_{date}: {e}")

# 執行繪圖
plotter = ArduinoFeaturePlotter(arduino_features)  # 👈 這裡用 arduino_features
plotter.plot_all()




🎨 繪製 Arduino 特徵圖...
   💾 已儲存: reports/figures/arduino_features\pineapple_07_20250203_arduino.html
   💾 已儲存: reports/figures/arduino_features\pineapple_07_20250204_arduino.html
   💾 已儲存: reports/figures/arduino_features\pineapple_07_20250205_arduino.html
   💾 已儲存: reports/figures/arduino_features\pineapple_08_20250203_arduino.html
   💾 已儲存: reports/figures/arduino_features\pineapple_08_20250204_arduino.html
   💾 已儲存: reports/figures/arduino_features\pineapple_08_20250205_arduino.html
✅ Arduino 特徵圖繪製完成！


In [57]:
# ===== Cell 9: 分布與穩定度分析 =====

class DistributionAnalyzer:
    """感測器分布與穩定度分析"""
    
    def __init__(self, output_dir='reports/figures'):
        self.output_dir = output_dir
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
    
    def plot_sensor_distributions(self, data_dict, data_type='pineapple'):
        """繪製感測器數值分布"""
        print(f"\n📊 繪製 {data_type} 分布圖...")
        
        if data_type == 'air':
            # Air: 每個 key 一張圖（key = 鳳梨編號_日期）
            for pid, df in data_dict.items():
                self._plot_single_distribution(df, f"Air_{pid}", 
                    f"{self.output_dir}/distributions/air_{pid}_dist.html")
        else:
            # Pineapple: 每個 ID+date 一張圖
            for pid, date_dict in data_dict.items():
                for date, df in date_dict.items():
                    self._plot_single_distribution(df, f"Pineapple_{pid}_{date}",
                        f"{self.output_dir}/distributions/pineapple_{pid}_{date}_dist.html")
        
        print("✅ 分布圖繪製完成！")
    
    def _plot_single_distribution(self, df, title, save_path):
        """繪製單一數據集的分布"""
        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=['MQ2', 'MQ3', 'MQ9', 'MQ135', 'TGS2602', 'Statistics'],
            specs=[[{"type": "box"}, {"type": "box"}, {"type": "box"}],
                   [{"type": "box"}, {"type": "box"}, {"type": "table"}]]
        )
        
        colors = ['blue', 'red', 'green', 'purple', 'orange']
        stats_data = []
        
        for idx, (col, color) in enumerate(zip(self.sensor_cols, colors)):
            if col in df.columns:
                row = (idx // 3) + 1
                col_pos = (idx % 3) + 1
                
                # Box plot
                fig.add_trace(
                    go.Box(
                        y=df[col],
                        name=col.replace('_raw', ''),
                        marker_color=color,
                        boxmean='sd',
                        showlegend=False
                    ),
                    row=row, col=col_pos
                )
                
                # 計算統計量
                stats_data.append([
                    col.replace('_raw', ''),
                    f"{df[col].mean():.2f}",
                    f"{df[col].std():.2f}",
                    f"{df[col].min():.0f}",
                    f"{df[col].max():.0f}",
                    f"{df[col].std()/df[col].mean()*100:.2f}%"
                ])
        
        # 統計表格
        fig.add_trace(
            go.Table(
                header=dict(
                    values=['Sensor', 'Mean', 'Std', 'Min', 'Max', 'CV%'],
                    fill_color='lightgray',
                    align='left'
                ),
                cells=dict(
                    values=list(zip(*stats_data)),
                    fill_color='white',
                    align='left'
                )
            ),
            row=2, col=3
        )
        
        fig.update_layout(
            title_text=f"Sensor Distributions - {title}",
            height=600,
            showlegend=False,
            template='plotly_white'
        )
        
        fig.write_html(save_path)
    
    def calculate_stability_scores(self, data_dict, data_type='pineapple'):
        """計算穩定度分數（CV: Coefficient of Variation）"""
        print(f"\n🎯 計算 {data_type} 穩定度分數...")
        
        stability_report = {}
        
        if data_type == 'air':
            for pid, df in data_dict.items():
                stability_report[pid] = self._calc_single_stability(df)
        else:
            for pid, date_dict in data_dict.items():
                stability_report[pid] = {}
                for date, df in date_dict.items():
                    stability_report[pid][date] = self._calc_single_stability(df)
        
        return stability_report
    
    def _calc_single_stability(self, df):
        """計算單一數據集的穩定度"""
        stability = {}
        
        for col in self.sensor_cols:
            if col in df.columns:
                mean_val = df[col].mean()
                std_val = df[col].std()
                cv = (std_val / mean_val * 100) if mean_val != 0 else np.inf
                
                stability[col] = {
                    'mean': mean_val,
                    'std': std_val,
                    'cv_percent': cv,
                    'stability_grade': self._grade_stability(cv)
                }
        
        return stability
    
    def _grade_stability(self, cv):
        """根據 CV 評分穩定度"""
        if cv < 5:
            return 'A (Excellent)'
        elif cv < 10:
            return 'B (Good)'
        elif cv < 20:
            return 'C (Fair)'
        else:
            return 'D (Poor)'

# 執行分布分析
dist_analyzer = DistributionAnalyzer()
dist_analyzer.plot_sensor_distributions(cleaned_air, 'air')
dist_analyzer.plot_sensor_distributions(cleaned_pineapple, 'pineapple')

# 計算穩定度
air_stability = dist_analyzer.calculate_stability_scores(cleaned_air, 'air')
pineapple_stability = dist_analyzer.calculate_stability_scores(cleaned_pineapple, 'pineapple')


📊 繪製 air 分布圖...
✅ 分布圖繪製完成！

📊 繪製 pineapple 分布圖...
✅ 分布圖繪製完成！

🎯 計算 air 穩定度分數...

🎯 計算 pineapple 穩定度分數...


In [58]:
# ===== Cell 10: 相關性分析 =====

class CorrelationAnalyzer:
    """感測器間相關性分析"""
    
    def __init__(self, output_dir='reports/figures'):
        self.output_dir = output_dir
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
    
    def plot_correlation_matrix(self, df, title, save_path):
        """繪製相關性矩陣熱力圖"""
        # 只選擇感測器欄位
        sensor_data = df[self.sensor_cols]
        
        # 計算相關係數
        corr_matrix = sensor_data.corr()
        
        # 繪製熱力圖
        fig = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=[col.replace('_raw', '') for col in corr_matrix.columns],
            y=[col.replace('_raw', '') for col in corr_matrix.index],
            colorscale='RdBu',
            zmid=0,
            text=corr_matrix.values,
            texttemplate='%{text:.2f}',
            textfont={"size": 12},
            colorbar=dict(title="Correlation")
        ))
        
        fig.update_layout(
            title=f"Sensor Correlation Matrix - {title}",
            xaxis_title="Sensors",
            yaxis_title="Sensors",
            height=500,
            width=600,
            template='plotly_white'
        )
        
        fig.write_html(save_path)
        print(f"   💾 已儲存: {save_path}")
    
    def analyze_all_correlations(self, cleaned_air, cleaned_pineapple):
        """批次分析所有相關性"""
        print("\n🔗 繪製相關性矩陣...")
        
        # Air
        for pid, df in cleaned_air.items():
            save_path = f"{self.output_dir}/correlation/air_{pid}_corr.html"
            self.plot_correlation_matrix(df, f"Air_{pid}", save_path)
        
        # Pineapple
        for pid, date_dict in cleaned_pineapple.items():
            for date, df in date_dict.items():
                save_path = f"{self.output_dir}/correlation/pineapple_{pid}_{date}_corr.html"
                self.plot_correlation_matrix(df, f"Pineapple_{pid}_{date}", save_path)
        
        print("✅ 相關性分析完成！")

# 執行相關性分析
corr_analyzer = CorrelationAnalyzer()
corr_analyzer.analyze_all_correlations(cleaned_air, cleaned_pineapple)


🔗 繪製相關性矩陣...
   💾 已儲存: reports/figures/correlation/air_shared_20250203_corr.html
   💾 已儲存: reports/figures/correlation/air_shared_20250204_corr.html
   💾 已儲存: reports/figures/correlation/air_shared_20250205_corr.html
   💾 已儲存: reports/figures/correlation/pineapple_07_20250203_corr.html
   💾 已儲存: reports/figures/correlation/pineapple_07_20250204_corr.html
   💾 已儲存: reports/figures/correlation/pineapple_07_20250205_corr.html
   💾 已儲存: reports/figures/correlation/pineapple_08_20250203_corr.html
   💾 已儲存: reports/figures/correlation/pineapple_08_20250204_corr.html
   💾 已儲存: reports/figures/correlation/pineapple_08_20250205_corr.html
✅ 相關性分析完成！


In [59]:
# ===== Cell 11: 綜合摘要報告 =====

class SummaryReportGenerator:
    """生成每顆鳳梨的摘要報告"""
    
    def __init__(self, output_dir='reports/summary'):
        self.output_dir = output_dir
    
    def generate_full_report(self, cleaned_air, cleaned_pineapple, delta_data, 
                            cleaning_report, air_stability, pineapple_stability):
        """生成完整的摘要報告"""
        print("\n📝 生成摘要報告...")
        
        report = {
            'generation_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'overview': self._generate_overview(cleaned_air, cleaned_pineapple),
            'data_quality': self._generate_quality_report(cleaning_report),
            'stability_analysis': self._generate_stability_report(air_stability, pineapple_stability),
            'sensor_health': self._generate_sensor_health_summary(cleaning_report),  # 👈 新增
            'arduino_features': self._check_arduino_features(delta_data),  # 👈 新增
            'recommendations': self._generate_recommendations(cleaned_air, cleaned_pineapple, 
                                                             pineapple_stability, cleaning_report)
        }
        
        # 儲存 JSON
        json_path = f"{self.output_dir}/full_report.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(report, f, ensure_ascii=False, indent=2, default=str)
        
        # 生成人類可讀的報告
        self._generate_readable_report(report)
        
        print(f"✅ 報告已生成: {json_path}")
        return report
    
    def _generate_overview(self, cleaned_air, cleaned_pineapple):
        """生成總覽"""
        total_pineapples = len(cleaned_pineapple)
        total_measurements = sum(len(date_dict) for date_dict in cleaned_pineapple.values())
        
        return {
            'total_pineapples': total_pineapples,
            'total_measurements': total_measurements,
            'pineapple_ids': list(cleaned_pineapple.keys())
        }
    
    def _generate_quality_report(self, cleaning_report):
        """生成數據品質報告"""
        quality = {'air': {}, 'pineapple': {}}
        
        for data_type in ['air', 'pineapple']:
            for pid, info in cleaning_report[data_type].items():
                if data_type == 'air':
                    quality['air'][pid] = {
                        'data_retention_rate': f"{(1 - info['removed_rows']/info['original_rows'])*100:.2f}%",
                        'total_outliers': sum(info['outlier_counts'].values()) if info['outlier_counts'] else 0
                    }
                else:
                    quality['pineapple'][pid] = {}
                    for date, date_info in info.items():
                        quality['pineapple'][pid][date] = {
                            'data_retention_rate': f"{(1 - date_info['removed_rows']/date_info['original_rows'])*100:.2f}%",
                            'total_outliers': sum(date_info['outlier_counts'].values()) if date_info['outlier_counts'] else 0
                        }
        
        return quality
    
    def _generate_stability_report(self, air_stability, pineapple_stability):
        """生成穩定度報告"""
        return {
            'air': air_stability,
            'pineapple': pineapple_stability
        }
    
    def _generate_sensor_health_summary(self, cleaning_report):
        """👈 新增：生成感測器健康度摘要"""
        health_summary = {'air': {}, 'pineapple': {}}
        
        for data_type in ['air', 'pineapple']:
            for pid, info in cleaning_report[data_type].items():
                if data_type == 'air':
                    health_summary['air'][pid] = info.get('sensor_health', {})
                else:
                    health_summary['pineapple'][pid] = {}
                    for date, date_info in info.items():
                        health_summary['pineapple'][pid][date] = date_info.get('sensor_health', {})
        
        return health_summary
    
    def _check_arduino_features(self, delta_data):
        """👈 新增：檢查 Arduino 特徵可用性"""
        feature_status = {}
        
        for pid, date_dict in delta_data.items():
            feature_status[pid] = {}
            for date, delta_df in date_dict.items():
                has_features = delta_df.attrs.get('has_arduino_features', False)
                feature_status[pid][date] = has_features
        
        total = sum(len(dates) for dates in feature_status.values())
        available = sum(1 for dates in feature_status.values() for has in dates.values() if has)
        
        return {
            'availability': f"{available}/{total}",
            'percentage': f"{available/total*100:.1f}%" if total > 0 else "0%",
            'details': feature_status
        }
    
    def _generate_recommendations(self, cleaned_air, cleaned_pineapple, 
                                 pineapple_stability, cleaning_report):
        """生成建議（含感測器健康度）"""
        recommendations = []
        
        # 檢查數據量
        total_pineapples = len(cleaned_pineapple)
        if total_pineapples < 10:
            recommendations.append({
                'type': 'data_sufficiency',
                'priority': 'high',
                'message': f"目前只有 {total_pineapples} 顆鳳梨，建議追加至少 10 顆以上以提高模型可靠性"
            })
        
        # 檢查穩定度
        for pid, date_dict in pineapple_stability.items():
            for date, stability in date_dict.items():
                poor_sensors = [sensor for sensor, info in stability.items() 
                               if 'Poor' in info['stability_grade']]
                if poor_sensors:
                    recommendations.append({
                        'type': 'sensor_stability',
                        'priority': 'medium',
                        'pineapple': f"{pid}_{date}",
                        'message': f"感測器 {poor_sensors} 穩定度較差（CV > 20%）"
                    })
        
        # 👈 新增：檢查感測器健康度
        for pid, info in cleaning_report['pineapple'].items():
            for date, date_info in info.items():
                health = date_info.get('sensor_health', {})
                for sensor, sensor_health in health.items():
                    if sensor_health['cv_percent'] < 1:
                        recommendations.append({
                            'type': 'sensor_health',
                            'priority': 'low',
                            'pineapple': f"{pid}_{date}",
                            'message': f"{sensor} 變異極小（CV < 1%），可能未接觸氣體或感測器故障"
                        })
                    
                    if sensor_health['significant_jumps'] > 10:
                        recommendations.append({
                            'type': 'sensor_noise',
                            'priority': 'medium',
                            'pineapple': f"{pid}_{date}",
                            'message': f"{sensor} 有 {sensor_health['significant_jumps']} 次顯著跳變，檢查訊號品質"
                        })
        
        return recommendations
    
    def _generate_readable_report(self, report):
        """生成人類可讀的 Markdown 報告"""
        md_path = f"{self.output_dir}/summary_report.md"
        
        with open(md_path, 'w', encoding='utf-8') as f:
            f.write("# 鳳梨氣味感測器數據分析報告\n\n")
            f.write(f"**生成時間**: {report['generation_time']}\n\n")
            
            f.write("## 📊 數據總覽\n\n")
            f.write(f"- 鳳梨總數: {report['overview']['total_pineapples']} 顆\n")
            f.write(f"- 測量次數: {report['overview']['total_measurements']} 次\n")
            f.write(f"- 鳳梨編號: {', '.join(report['overview']['pineapple_ids'])}\n\n")
            
            # 👈 新增：Arduino 特徵可用性
            f.write("## 🎨 Arduino 預處理特徵\n\n")
            f.write(f"- 可用性: {report['arduino_features']['availability']} ({report['arduino_features']['percentage']})\n")
            if report['arduino_features']['percentage'] == '100.0%':
                f.write("- ✅ 所有數據集都包含完整的 Arduino 特徵（log ratio, delta, MA5）\n\n")
            else:
                f.write("- ⚠️  部分數據集缺少 Arduino 特徵\n\n")
            
            f.write("## ✅ 數據品質評估\n\n")
            f.write("### Air Baseline\n\n")
            for pid, quality in report['data_quality']['air'].items():
                f.write(f"- **Air_{pid}**: 保留率 {quality['data_retention_rate']}, "
                       f"異常值 {quality['total_outliers']} 個\n")
            
            f.write("\n### Pineapple 測量\n\n")
            for pid, dates in report['data_quality']['pineapple'].items():
                for date, quality in dates.items():
                    f.write(f"- **Pineapple_{pid}_{date}**: 保留率 {quality['data_retention_rate']}, "
                           f"異常值 {quality['total_outliers']} 個\n")
            
            f.write("\n## 💡 建議事項\n\n")
            if report['recommendations']:
                # 按優先級分組
                high_priority = [r for r in report['recommendations'] if r['priority'] == 'high']
                medium_priority = [r for r in report['recommendations'] if r['priority'] == 'medium']
                low_priority = [r for r in report['recommendations'] if r['priority'] == 'low']
                
                if high_priority:
                    f.write("### 🔴 高優先級\n\n")
                    for rec in high_priority:
                        f.write(f"- **{rec['type']}**: {rec['message']}\n")
                    f.write("\n")
                
                if medium_priority:
                    f.write("### 🟡 中優先級\n\n")
                    for rec in medium_priority:
                        pineapple = rec.get('pineapple', '')
                        f.write(f"- **{rec['type']}** ({pineapple}): {rec['message']}\n")
                    f.write("\n")
                
                if low_priority:
                    f.write("### 🟢 低優先級\n\n")
                    for rec in low_priority:
                        pineapple = rec.get('pineapple', '')
                        f.write(f"- **{rec['type']}** ({pineapple}): {rec['message']}\n")
                    f.write("\n")
            else:
                f.write("✅ 數據品質良好，無需額外處理\n\n")
        
        print(f"   💾 已生成 Markdown 報告: {md_path}")

# 生成報告
report_gen = SummaryReportGenerator()
final_report = report_gen.generate_full_report(
    cleaned_air, cleaned_pineapple, delta_data,
    cleaning_report, air_stability, pineapple_stability
)


📝 生成摘要報告...
   💾 已生成 Markdown 報告: reports/summary/summary_report.md
✅ 報告已生成: reports/summary/full_report.json


In [60]:
# ===== Cell 12: 最終總結 =====

print("\n" + "="*60)
print("🎉 Step 1: 自動化資料清洗與初步分析 (EDA) 完成！")
print("="*60)

print("\n📁 已生成的檔案：")
print("   📊 Raw 時序圖: reports/figures/raw_timeseries/")
print("   📉 Delta 時序圖: reports/figures/delta_timeseries/")
print("   🎨 Arduino 特徵圖: reports/figures/arduino_features/")
print("   📈 分布圖: reports/figures/distributions/")
print("   🔗 相關性矩陣: reports/figures/correlation/")
print("   📝 摘要報告: reports/summary/")

print("\n💡 關鍵發現：")
print(f"   • 鳳梨總數: {final_report['overview']['total_pineapples']} 顆")
print(f"   • 測量次數: {final_report['overview']['total_measurements']} 次")
print(f"   • Arduino 特徵可用性: {final_report['arduino_features']['percentage']}")

if final_report['recommendations']:
    print("\n⚠️  建議事項：")
    high_priority = [r for r in final_report['recommendations'] if r['priority'] == 'high']
    medium_priority = [r for r in final_report['recommendations'] if r['priority'] == 'medium']
    
    for rec in high_priority[:3]:  # 只顯示前3個高優先級
        print(f"   [高] {rec['message']}")
    
    for rec in medium_priority[:3]:  # 只顯示前3個中優先級
        print(f"   [中] {rec['message']}")
    
    if len(final_report['recommendations']) > 6:
        print(f"   ... 等共 {len(final_report['recommendations'])} 項建議（詳見報告）")

print("\n✅ 數據可用性評估：")
if final_report['overview']['total_pineapples'] >= 10:
    print("   ✓ 數據量充足，可進入 Step 3（成熟階段定義）")
else:
    print("   ⚠️  建議追加測試（Step 2），目標至少 10 顆鳳梨")

print("\n🚀 下一步 (Step 2)：")
print("   1. 檢視 reports/ 資料夾中的所有圖表")
print("   2. 特別注意 Arduino 特徵圖（arduino_features/）")
print("   3. 與小組討論數據品質與樣本數量")
print("   4. 決定是否需要追加測試")
print("   5. 確認後進入 Step 3: Change-point detection & Clustering")

print("\n📌 提醒：")
print("   • Arduino 已預處理的特徵（log ratio, delta, MA5）已完整保留")
print("   • 感測器健康度報告已整合到摘要中")
print("   • 所有視覺化圖表都是互動式 HTML，可用瀏覽器開啟")

print("\n" + "="*60)


🎉 Step 1: 自動化資料清洗與初步分析 (EDA) 完成！

📁 已生成的檔案：
   📊 Raw 時序圖: reports/figures/raw_timeseries/
   📉 Delta 時序圖: reports/figures/delta_timeseries/
   🎨 Arduino 特徵圖: reports/figures/arduino_features/
   📈 分布圖: reports/figures/distributions/
   🔗 相關性矩陣: reports/figures/correlation/
   📝 摘要報告: reports/summary/

💡 關鍵發現：
   • 鳳梨總數: 2 顆
   • 測量次數: 6 次
   • Arduino 特徵可用性: 0.0%

⚠️  建議事項：
   [高] 目前只有 2 顆鳳梨，建議追加至少 10 顆以上以提高模型可靠性
   ... 等共 15 項建議（詳見報告）

✅ 數據可用性評估：
   ⚠️  建議追加測試（Step 2），目標至少 10 顆鳳梨

🚀 下一步 (Step 2)：
   1. 檢視 reports/ 資料夾中的所有圖表
   2. 特別注意 Arduino 特徵圖（arduino_features/）
   3. 與小組討論數據品質與樣本數量
   4. 決定是否需要追加測試
   5. 確認後進入 Step 3: Change-point detection & Clustering

📌 提醒：
   • Arduino 已預處理的特徵（log ratio, delta, MA5）已完整保留
   • 感測器健康度報告已整合到摘要中
   • 所有視覺化圖表都是互動式 HTML，可用瀏覽器開啟

